In [ ]:
# Google Colaboratory를 사용할 때는 다음 주석을 해제하고 실행하기 바랍니다.

In [ ]:
# # 다음을 실행하면 authorization code 입력을 요청받습니다.
# # 출력된 링크를 클릭하고 Google 계정으로 로그인한 뒤,
# # authorization code를 복사해서 붙여 넣습니다.
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# working_dir = 'MLSys_100Knocks' #　※※ 여러분이 만든 폴더 경로가 다를 때는 다음을 변경합니다. ※※
# path = f'/content/drive/MyDrive/{working_dir}/MainChapter/chapter01'
# os.chdir(path)

# １장 분석을 위한 준비 테크닉 10

모 피자 체인의 매출 추이를 분석합니다.<br>
데이터의 내용을 이해하고 분석하기 용이한 형태로 가공하는 절차를 학습합니다.

### 테크닉 1: 데이터를 모두 로딩하자

In [ ]:
import pandas as pd
m_store = pd.read_csv('m_store.csv')
m_store

In [ ]:
len(m_store)

In [ ]:
m_store.head()

In [ ]:
m_area = pd.read_csv('m_area.csv')
m_area

In [ ]:
tbl_order_4 = pd.read_csv('tbl_order_202104.csv')
tbl_order_4

### 테크닉 2: 데이터를 결합(유니온)하자

In [ ]:
tbl_order_5 = pd.read_csv('tbl_order_202105.csv')
tbl_order_5

In [ ]:
order_all = pd.concat([tbl_order_4, tbl_order_5], ignore_index=True)
order_all

In [ ]:
len(order_all) == len(tbl_order_4) + len(tbl_order_5)

### 테크닉 3: 폴더 안에 있는 파일을 확인하자

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
# 모든 파일 목록을 표시한다
os.listdir(current_dir)

In [ ]:
tbl_order_file = os.path.join(current_dir, 'tbl_order_*.csv')
tbl_order_file

In [ ]:
# 특정 파일 목록을 표시한다
import glob
tbl_order_files = glob.glob(tbl_order_file)
tbl_order_files

### 테크닉 4: 여러 데이터를 결합(유니온)하자

In [ ]:
# 반복 전에 1개만 처리해서 확인한다
order_all = pd.DataFrame()
file = tbl_order_files[0]
order_data = pd.read_csv(file)
print(f'{file}:{len(order_data)}')
order_all = pd.concat([order_all, order_data], ignore_index=True)
order_all

In [ ]:
# 반복 처리
order_all = pd.DataFrame()
for file in tbl_order_files:
    order_data = pd.read_csv(file)
    print(f'{file}:{len(order_data)}')
    order_all = pd.concat([order_all, order_data], ignore_index=True)

In [ ]:
order_all

### 테크닉 5: 데이터 통계량을 확인해 보자

In [ ]:
order_all.isnull().sum()

In [ ]:
# pd.options.display.float_format = '{:.3f}'.format  # 소수점이하 3자리로 표시
order_all.describe()

In [ ]:
order_all['total_amount'].describe()

In [ ]:
print(order_all['order_accept_date'].min())
print(order_all['order_accept_date'].max())
print(order_all['delivered_date'].min())
print(order_all['delivered_date'].max())

### 테크닉 6: 불필요한 데이터를 제거하자

In [ ]:
order_data = order_all.loc[order_all['store_id'] != 999]
order_data

### 테크닉 7: 마스터 데이터를 결합(조인)하자

In [ ]:
order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data

In [ ]:
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')
order_data

### 테크닉 8: 마스터가 존재하지 않는 코드에 이름을 설정하자

In [ ]:
order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'delivery'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'takeout'
order_data

In [ ]:
order_data.loc[order_data['status'] == 0, 'status_name'] = '주문 접수'
order_data.loc[order_data['status'] == 1, 'status_name'] = '결제 완료'
order_data.loc[order_data['status'] == 2, 'status_name'] = '배달 완료'
order_data.loc[order_data['status'] == 9, 'status_name'] = '주문 취소'
order_data

### 테크닉 9: 분석 기초 테이블을 출력하자

In [ ]:
output_dir = os.path.join(current_dir, 'output_data')
os.makedirs(output_dir, exist_ok=True)

In [ ]:
output_file = os.path.join(output_dir, 'order_data.csv')
order_data.to_csv(output_file, index=False)

### 테크닉 10: 셀을 사용하기 쉽게 정리하자

In [ ]:
# 라이브러리 임포트
import pandas as pd
import os
import glob

# 파일 로딩
m_store = pd.read_csv('m_store.csv')
m_area = pd.read_csv('m_area.csv')

# 주문 데이터 로딩
current_dir = os.getcwd()
tbl_order_file = os.path.join(current_dir, 'tbl_order_*.csv')
tbl_order_files = glob.glob(tbl_order_file)
order_all = pd.DataFrame()

for file in tbl_order_files:
    order_data = pd.read_csv(file)
    print(f'{file}:{len(order_data)}')
    order_all = pd.concat([order_all, order_data], ignore_index=True)

# 불필요한 데이터 제거
order_data = order_all.loc[order_all['store_id'] != 999]

# 마스터 데이터 결합
order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')

# 이름 설정(수령 방법)
order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'delivery'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'takeout'

# 이름 설정(주문 상태)
order_data.loc[order_data['status'] == 0, 'status_name'] = '주문 접수'
order_data.loc[order_data['status'] == 1, 'status_name'] = '결제 완료'
order_data.loc[order_data['status'] == 2, 'status_name'] = '배달 완료'
order_data.loc[order_data['status'] == 9, 'status_name'] = '주문 취소'

# 파일에 저장
output_dir = os.path.join(current_dir, 'output_data')
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'order_data.csv')
order_data.to_csv(output_file, index=False)